# Hybrid Quantum-Classical Time Series Forecasting

This notebook builds a hybrid quantum-classical model for time series forecasting on a synthetic dataset and compares it with a classical baseline.


## 1. Setup
Install required libraries and configure the environment.


In [ ]:
!pip -q install pennylane torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu > /dev/null


In [ ]:
import math
import random
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

plt.style.use('seaborn-v0_8')

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


## 2. Data Preparation
Generate a synthetic time series and prepare training/test windows.


In [ ]:
# Generate a synthetic time series that combines trend, seasonal, and noise components
n_points = 1800
time = np.arange(n_points)
series = 0.05 * time + np.sin(0.2 * time) + 0.5 * np.sin(0.05 * time) + 0.3 * np.cos(0.15 * time)
noise = np.random.normal(scale=0.2, size=n_points)
series += noise

series_df = pd.DataFrame({"t": time, "value": series})
series_df.head()


In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(series_df["t"], series_df["value"], label="Synthetic series")
ax.set_xlabel("Time step")
ax.set_ylabel("Value")
ax.set_title("Synthetic time series used for forecasting")
ax.legend()
plt.show()


In [ ]:
# Scaling
scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series_df[["value"]]).flatten()

# Windowing helper
def create_windows(data, window_size, horizon=1):
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i : i + window_size])
        y.append(data[i + window_size : i + window_size + horizon])
    return np.array(X), np.array(y)

window_size = 32
horizon = 1
X, y = create_windows(series_scaled, window_size, horizon)

split_idx = int(0.75 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

batch_size = 64
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size)

X_train_tensor.shape, y_train_tensor.shape


## 3. Classical Baseline (LSTM)
Train a purely classical LSTM baseline.


In [ ]:
class LSTMForecaster(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=1, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, horizon)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_hidden = lstm_out[:, -1, :]
        return self.fc(last_hidden)


def train_model(model, train_loader, val_loader, epochs=30, lr=1e-3):
    device = torch.device("cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    history = {"train_loss": [], "val_loss": []}

    for epoch in range(1, epochs + 1):
        model.train()
        train_losses = []
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                loss = criterion(preds, yb)
                val_losses.append(loss.item())

        history["train_loss"].append(np.mean(train_losses))
        history["val_loss"].append(np.mean(val_losses))
        if epoch % 5 == 0 or epoch == 1:
            print(f"Epoch {epoch:02d} | Train Loss: {history['train_loss'][-1]:.4f} | Val Loss: {history['val_loss'][-1]:.4f}")

    return history


In [ ]:
baseline_model = LSTMForecaster(hidden_size=32)
baseline_history = train_model(baseline_model, train_loader, val_loader, epochs=35, lr=5e-3)


In [ ]:
def evaluate(model, X_tensor, y_true_tensor):
    model.eval()
    with torch.no_grad():
        preds = model(X_tensor).cpu().numpy().flatten()
    y_true = y_true_tensor.cpu().numpy().flatten()
    mse = mean_squared_error(y_true, preds)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(y_true, preds)
    return preds, {"MSE": mse, "RMSE": rmse, "MAE": mae}

baseline_preds, baseline_metrics = evaluate(baseline_model, X_test_tensor, y_test_tensor)
baseline_metrics


In [ ]:
# Plot baseline predictions vs actuals on the test set
actual = scaler.inverse_transform(y_test)
predicted = scaler.inverse_transform(baseline_preds.reshape(-1, 1))

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(actual, label="Actual", linewidth=2)
ax.plot(predicted, label="LSTM Baseline", linestyle='--')
ax.set_title("Baseline LSTM Forecast vs Actual")
ax.set_xlabel("Test time step")
ax.set_ylabel("Value")
ax.legend()
plt.show()


## 4. Hybrid Quantum-Classical Model
Build an LSTM combined with a variational quantum circuit layer implemented with PennyLane.


In [ ]:
class QuantumLSTMForecaster(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, n_qubits=4, q_depth=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.n_qubits = n_qubits

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.to_qubits = nn.Linear(hidden_size, n_qubits)

        dev = qml.device("default.qubit.torch", wires=n_qubits)

        @qml.qnode(dev, interface="torch")
        def circuit(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')
            qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
            return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

        weight_shapes = {"weights": (q_depth, n_qubits, 3)}
        self.quantum_layer = qml.qnn.TorchLayer(circuit, weight_shapes)
        self.readout = nn.Sequential(
            nn.Linear(n_qubits, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, horizon)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_hidden = lstm_out[:, -1, :]
        qubit_inputs = torch.tanh(self.to_qubits(last_hidden))
        quantum_out = self.quantum_layer(qubit_inputs)
        return self.readout(quantum_out)


In [ ]:
quantum_model = QuantumLSTMForecaster(hidden_size=32, n_qubits=4, q_depth=2)
quantum_history = train_model(quantum_model, train_loader, val_loader, epochs=35, lr=5e-3)


In [ ]:
quantum_preds, quantum_metrics = evaluate(quantum_model, X_test_tensor, y_test_tensor)
quantum_metrics


In [ ]:
actual = scaler.inverse_transform(y_test)
quantum_predicted = scaler.inverse_transform(quantum_preds.reshape(-1, 1))

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(actual, label="Actual", linewidth=2)
ax.plot(quantum_predicted, label="Hybrid QNN", linestyle='--')
ax.set_title("Hybrid Quantum-Classical Forecast vs Actual")
ax.set_xlabel("Test time step")
ax.set_ylabel("Value")
ax.legend()
plt.show()


## 5. Circuit Depth Experiments
Evaluate different quantum circuit depths and compare metrics.


In [ ]:
results = []
for depth in [1, 2, 3]:
    print(f"\nTraining hybrid model with circuit depth = {depth}")
    model = QuantumLSTMForecaster(hidden_size=32, n_qubits=4, q_depth=depth)
    train_model(model, train_loader, val_loader, epochs=25, lr=5e-3)
    _, metrics = evaluate(model, X_test_tensor, y_test_tensor)
    results.append({"Circuit depth": depth, **metrics})

depth_results_df = pd.DataFrame(results)
depth_results_df


## 6. Comparison Summary


In [ ]:
comparison_df = pd.DataFrame([
    {"Model": "LSTM Baseline", **baseline_metrics},
    {"Model": "Hybrid QNN (depth=2)", **quantum_metrics}
])
comparison_df


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
metrics_to_plot = ["RMSE", "MAE"]
width = 0.35
x = np.arange(len(metrics_to_plot))

baseline_vals = [baseline_metrics[m] for m in metrics_to_plot]
quantum_vals = [quantum_metrics[m] for m in metrics_to_plot]

ax.bar(x - width/2, baseline_vals, width, label="LSTM Baseline")
ax.bar(x + width/2, quantum_vals, width, label="Hybrid QNN")
ax.set_xticks(x)
ax.set_xticklabels(metrics_to_plot)
ax.set_ylabel("Error")
ax.set_title("Error comparison")
ax.legend()
plt.show()


The notebook above can be extended further by adjusting hyperparameters, experimenting with different quantum circuits, or changing the dataset.
